# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)
#### Aluno: Bruno Crespo Ferreira

## Problema

A cidade de União Paulista enfrenta uma séria carência orçamentária. Em busca de uma solução de longo prazo, a câmara de vereadores da cidade aprova uma melhoria da base da cobrança de impostos que prevê a condenação de uma área habitacional do centro da cidade e sua substituição por um conjunto habitacional moderno.

O projeto envolve duas fases:
1. demolição das casas que estão aquém do padrão para liberar terreno para o novo projeto;
2. construção do novo conjunto urbano. A seguir daremos um resumo da situação.

Um total de 300 casas aquém do padrão podem ser demolidas. Cada casa ocupa um lote de 0,25 acres. O custo de demolição de uma casa condenada é de \$2.000. Os tamanhos de lotes para domicílios (unidades) simples, duplos, triplos e quádruplos são de 0,18; 0,28; 0,4 e 0,5 acres, respectivamente. Ruas, espaços abertos e instalações públicas ocupam 15% da área disponível.

No novo conjunto habitacional, as unidades triplas e quádruplas representam no mínimo 25% do total. Unidades simples devem representar no mínimo 20% de todas as unidades, e unidades duplas, no mínimo 10%. O imposto cobrado por unidade para unidades simples, duplas, triplas e quádruplas é de \$1.000, \$1.900, \$2.700 e \$3.400, respectivamente.

O custo de construção por unidade domiciliar simples, dupla, tripla e quádrupla é de \$50.000, \$70.000, \$130.000 e \$160.000, respectivamente. O financiamento acordado com um banco local será de no máximo \$15 milhões.

### Objetivo

Quantas unidades de cada tipo devem ser construídas para maximizar a arrecadação de impostos?

## Instalação da biblioteca PuLP

In [1]:
!pip install pulp
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 43.9 MB/s eta 0:00:00


## Parâmetros

* $D$: conjunto dos tipos de domicílios.
* $I_{i} \in \mathbb{R}_+$: imposto cobrado para o domicílio do tipo $i \in D$.
* $lote\_casa \in \mathbb{R}_+$: ocupação (em acres) de uma casa aquém do padrão.
* $custo\_demolicao \in \mathbb{R}_+$: custo de demolição de uma casa aquém do padrão.
* $total\_casas \in \mathbb{N}$: total de casas aquém do padrão que podem ser demolidas.
* $l_{i} \in \mathbb{R}_+$: tamanho do lote necessário para construir um domicílio do tipo $i \in D$.
* $c_{i} \in \mathbb{R}_+$: custo de construção para uma unidade do domicílio do tipo $i \in D$.
* $financiamento$: valor máximo de financiamento que o projeto possui.

In [2]:
domicilios = ['simples', 'duplos', 'triplos', 'quadruplos']

imposto_por_domicilio = {
    'simples': 1000.0,
    'duplos': 1900.0,
    'triplos': 2700.0,
    'quadruplos': 3400.0
}

lote_casa = 0.25

custo_demolicao = 2000.0

total_casas = 300

lote_por_domicilio = {
    'simples': 0.18,
    'duplos': 0.28,
    'triplos': 0.40,
    'quadruplos': 0.50
}

custos_construcao = {
    'simples': 50000.0,
    'duplos': 70000.0,
    'triplos': 130000.0,
    'quadruplos': 160000.0
}

financiamento = 15000000.0

## Declaração do objeto que representa o modelo matemático



In [3]:
modelo = pulp.LpProblem('projeto_novo_habitacional', pulp.LpMaximize)

## Variáveis de decisão

* $x_{i} \ge 0$: quantidade de domicílios construidos do tipo $i \in D$.
* $a$: quantidade de casas aquém do padrão que serão demolidas.

In [4]:
domicilios_vars = pulp.LpVariable.dicts('domicilio', domicilios, cat='Integer', lowBound=0)
qtd_casas_demolidas = pulp.LpVariable('a', cat='Integer', lowBound=0)

## Função objetivo

* Maximizar a arrecadação de impostos do novo conjunto habitacional: $\max \sum_{i \in D} I_{i}x_{i}$


In [5]:
modelo += pulp.lpSum([imposto_por_domicilio[domicilio] * domicilios_vars[domicilio] for domicilio in domicilios])

## Restrições

* A quantidade de casas demolidas aquém do padrão deve respeitar o limite de casas que podem ser demolidas: $a \le total\_casas$

Observação: o enunciado dá a entender que nem todas as casa aquém do padrão serão demolidas, pois "Um total de 300 casas aquém do padrão **PODEM** ser demolidas."

In [6]:
modelo += qtd_casas_demolidas <= total_casas

* Os custos de construção de domicílios e os custos de demolição de casas aquém do padrão não podem ultrapassar o financiamento do projeto: $\sum_{i \in D} c_{i}x_{i} + custo\_demolicao * a \le financiamento$

In [7]:
modelo += pulp.lpSum([custos_construcao[domicilio] * domicilios_vars[domicilio] for domicilio in domicilios]) + \
custo_demolicao * qtd_casas_demolidas <= financiamento

* Deve-se respeitar as porcentagens de ocupação da nova área disponível, sendo que $15\%$ já estão reservadas: $\sum_{i \in D} l_{i}x_{i} \le 0.85(lote\_casa * a)$

In [8]:
modelo += pulp.lpSum([lote_por_domicilio[domicilio] * domicilios_vars[domicilio] for domicilio in domicilios]) <= 0.85 * (lote_casa * qtd_casas_demolidas)

Deve-se respeitar as proporções entre as quantidades de domicílios:

* Unidades simples devem representar pelo menos $20\%$ do total: $x_{1} \ge 0.20(\sum_{i \in D} x_{i})$
* Unidades duplas devem representar pelo menos $10\%$ do total: $x_{2} \ge 0.10(\sum_{i \in D} x_{i})$
* Unidades triplas e quádruplas devem representar pelo menos $25\%$ do total: $x_{3} + x_{4} \ge 0.25(\sum_{i \in D} x_{i})$

In [9]:
# domicílio simples
modelo += domicilios_vars['simples'] >= 0.20 * pulp.lpSum([domicilios_vars[domicilio] for domicilio in domicilios])

# domicílio duplo
modelo += domicilios_vars['duplos'] >= 0.10 * pulp.lpSum([domicilios_vars[domicilio] for domicilio in domicilios])

# domicílio triplo e quádruplo
modelo += domicilios_vars['triplos'] + domicilios_vars['quadruplos'] >= 0.25 * pulp.lpSum([domicilios_vars[domicilio] for domicilio in domicilios])

## Resolvendo o problema

* Mostrando os modelos (debug)

In [10]:
modelo

projeto_novo_habitacional:
MAXIMIZE
1900.0*domicilio_duplos + 3400.0*domicilio_quadruplos + 1000.0*domicilio_simples + 2700.0*domicilio_triplos + 0.0
SUBJECT TO
_C1: a <= 300

_C2: 2000 a + 70000 domicilio_duplos + 160000 domicilio_quadruplos
 + 50000 domicilio_simples + 130000 domicilio_triplos <= 15000000

_C3: - 0.2125 a + 0.28 domicilio_duplos + 0.5 domicilio_quadruplos
 + 0.18 domicilio_simples + 0.4 domicilio_triplos <= 0

_C4: - 0.2 domicilio_duplos - 0.2 domicilio_quadruplos + 0.8 domicilio_simples
 - 0.2 domicilio_triplos >= 0

_C5: 0.9 domicilio_duplos - 0.1 domicilio_quadruplos - 0.1 domicilio_simples
 - 0.1 domicilio_triplos >= 0

_C6: - 0.25 domicilio_duplos + 0.75 domicilio_quadruplos
 - 0.25 domicilio_simples + 0.75 domicilio_triplos >= 0

VARIABLES
0 <= a Integer
0 <= domicilio_duplos Integer
0 <= domicilio_quadruplos Integer
0 <= domicilio_simples Integer
0 <= domicilio_triplos Integer

* Resolvendo o modelo

In [11]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [12]:
print('Status:', pulp.LpStatus[status])
print('Função objetivo:', f'É possível arrecadar ${modelo.objective.value()} de impostos no melhor caso.')

print('\n\t\tSoluções')
print('Quantidade de casa aquém do padrão destruidas:', qtd_casas_demolidas.value())
print(f'Total da área do novo conjunto habitacional: {lote_casa * qtd_casas_demolidas.value()} acres.')
print(f'Total da área para construção dos domicílios: {0.85 * lote_casa * qtd_casas_demolidas.value()} acres.\n')

for domicilio in domicilios_vars:
  print(f'Quantidade de domicilios do tipo \'{domicilio}\' construídos:', domicilios_vars[domicilio].value())
  print(f'Área: {lote_por_domicilio[domicilio] * domicilios_vars[domicilio].value()} acres.\n')

Status: Optimal
Função objetivo: É possível arrecadar $343700.0 de impostos no melhor caso.

		Soluções
Quantidade de casa aquém do padrão destruidas: 245.0
Total da área do novo conjunto habitacional: 61.25 acres.
Total da área para construção dos domicílios: 52.0625 acres.

Quantidade de domicilios do tipo 'simples' construídos: 36.0
Área: 6.4799999999999995 acres.

Quantidade de domicilios do tipo 'duplos' construídos: 98.0
Área: 27.44 acres.

Quantidade de domicilios do tipo 'triplos' construídos: 45.0
Área: 18.0 acres.

Quantidade de domicilios do tipo 'quadruplos' construídos: 0.0
Área: 0.0 acres.

